In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [37]:
import mysql.connector
from mysql.connector import Error

try:
    # Membuat koneksi ke database MySQL
    connection = mysql.connector.connect(
        host='localhost',          # Nama host atau IP server MySQL
        database='PerpustakaanDigital',  # Nama database yang akan digunakan
        user='root',               # Username MySQL
        password=''        # Password MySQL
    )
    
    if connection.is_connected():
        print("Koneksi ke MySQL berhasil!")       
        # Membuat cursor untuk eksekusi query
        cursor = connection.cursor()
        
        # Menjalankan query SQL
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"Anda terhubung ke database: {record[0]}")

        # Menampilkan semua data anggota
        print("\nData Anggota:")
        cursor.execute("SELECT * FROM Anggota;")
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Update Status Peminjaman yang Sudah Lewat Tanggal Kembali
        cursor.execute("UPDATE Peminjaman SET Status = 'Overdue' WHERE Tanggal_Kembali < CURDATE();")
        connection.commit()
        print("\nStatus peminjaman yang sudah lewat tanggal kembali telah diperbarui.")

        # Menambahkan 10 Eksemplar Buku "Pemrograman Python"
        cursor.execute("UPDATE Buku SET Jumlah = Jumlah + 10 WHERE Judul_Buku = 'Pemrograman Python';")
        connection.commit()
        print("\n10 eksemplar buku 'Pemrograman Python' telah ditambahkan.")

        # Hapus Data Anggota yang Tidak Aktif Selama Lebih dari Satu Tahun
        cursor.execute("""
        DELETE FROM Anggota
        WHERE ID_Anggota NOT IN (
            SELECT DISTINCT ID_Anggota
            FROM Peminjaman
            WHERE Tanggal_Pinjam > DATE_SUB(CURDATE(), INTERVAL 1 YEAR)
        );
        """)
        connection.commit()
        print("\nData anggota yang tidak aktif selama lebih dari satu tahun telah dihapus.")

        # Menampilkan data tabel Petugas
        print("\nData Petugas:")
        cursor.execute("SELECT * FROM Petugas;")
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Menampilkan judul buku dan pengarang dari tabel Buku yang diterbitkan setelah tahun 2015
        print("\nJudul Buku dan Pengarang (Tahun Terbit > 2015):")
        cursor.execute("SELECT Judul_Buku, Pengarang FROM Buku WHERE Tahun_Terbit > 2015;")
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Menampilkan nama anggota yang memiliki jumlah peminjaman lebih dari 3 buku
        print("\nNama Anggota dengan Peminjaman > 3 buku:")
        cursor.execute("""
        SELECT Nama
        FROM Anggota
        WHERE ID_Anggota IN (
            SELECT ID_Anggota
            FROM Peminjaman
            JOIN Detail_Peminjaman ON Peminjaman.ID_Peminjaman = Detail_Peminjaman.ID_Peminjaman
            GROUP BY ID_Anggota
            HAVING SUM(Jumlah_Buku) > 3
        );
        """)
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Buku yang paling banyak dipinjam
        print("\nBuku yang Paling Banyak Dipinjam:")
        cursor.execute("""
        SELECT Judul_Buku
        FROM Buku
        WHERE ID_Buku = (
            SELECT ID_Buku
            FROM Detail_Peminjaman
            GROUP BY ID_Buku
            ORDER BY SUM(Jumlah_Buku) DESC
            LIMIT 1
        );
        """)
        result = cursor.fetchone()
        print(result)

        # Menampilkan nama anggota, judul buku yang dipinjam, dan tanggal peminjaman
        print("\nNama Anggota, Judul Buku, dan Tanggal Peminjaman:")
        cursor.execute("""
        SELECT A.Nama, B.Judul_Buku, P.Tanggal_Pinjam
        FROM Anggota A
        JOIN Peminjaman P ON A.ID_Anggota = P.ID_Anggota
        JOIN Detail_Peminjaman DP ON P.ID_Peminjaman = DP.ID_Peminjaman
        JOIN Buku B ON DP.ID_Buku = B.ID_Buku;
        """)
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Petugas yang mengelola peminjaman terakhir
        print("\nPetugas yang Mengelola Peminjaman Terakhir:")
        cursor.execute("""
        SELECT Petugas.Nama_Petugas
        FROM Petugas
        JOIN Peminjaman ON Petugas.ID_Petugas = Peminjaman.ID_Anggota
        ORDER BY Tanggal_Pinjam DESC
        LIMIT 1;
        """)
        result = cursor.fetchone()
        print(result)

        # Total buku yang ada di perpustakaan
        print("\nTotal Buku di Perpustakaan:")
        cursor.execute("SELECT SUM(Jumlah) AS Total_Buku FROM Buku;")
        result = cursor.fetchone()
        print(result)

        # Rata-rata jumlah buku yang dipinjam per anggota
        print("\nRata-rata Jumlah Buku yang Dipinjam per Anggota:")
        cursor.execute("SELECT AVG(Jumlah_Buku) AS Rata2_Buku_Dipinjam FROM Detail_Peminjaman;")
        result = cursor.fetchone()
        print(result)

        # Kelompok buku berdasarkan kategori dan hitung jumlah buku di setiap kategori
        print("\nJumlah Buku per Kategori:")
        cursor.execute("""
        SELECT Kategori, SUM(Jumlah) AS Jumlah_Buku
        FROM Buku
        GROUP BY Kategori;
        """)
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Kelompok anggota berdasarkan Alamat dan hitung jumlah anggota per kota
        print("\nJumlah Anggota per Kota:")
        cursor.execute("""
        SELECT Alamat, COUNT(*) AS Jumlah_Anggota
        FROM Anggota
        GROUP BY Alamat;
        """)
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Menampilkan Nama Anggota dalam Huruf Kapital
        print("\nNama Anggota dalam Huruf Kapital:")
        cursor.execute("SELECT UPPER(Nama) AS Nama_Kapital FROM Anggota;")
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Menemukan Anggota yang Memiliki Nama Belakang "Putra"
        print("\nAnggota dengan Nama Belakang 'Putra':")
        cursor.execute("SELECT Nama FROM Anggota WHERE Nama LIKE '%Putra';")
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Menghitung Usia Setiap Anggota Berdasarkan Tanggal Lahir
        print("\nUsia Setiap Anggota:")
        cursor.execute("""
        SELECT Nama, Tanggal_Lahir, TIMESTAMPDIFF(YEAR, Tanggal_Lahir, CURDATE()) AS Usia
        FROM Anggota;
        """)
        result = cursor.fetchall()
        for row in result:
            print(row)

        # Menampilkan Peminjaman yang Dilakukan dalam 30 Hari Terakhir
        print("\nPeminjaman dalam 30 Hari Terakhir:")
        cursor.execute("SELECT * FROM Peminjaman WHERE Tanggal_Pinjam >= CURDATE() - INTERVAL 30 DAY;")
        result = cursor.fetchall()
        for row in result:
            print(row)

except Error as e:
    print(f"Error saat terhubung ke MySQL: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Koneksi ke MySQL ditutup.")


Koneksi ke MySQL berhasil!
Anda terhubung ke database: perpustakaandigital

Data Anggota:
(1, 'Ahmad Fauzi', 'Jl. Anggrek No.5', datetime.date(1985, 6, 10), 'ahmad@mail.com')
(2, 'Budi Putra', 'Jl. Melati No. 3', datetime.date(1990, 2, 18), 'budi@mail.com')
(4, 'Dian Sari', 'Jl. Kamboja No. 11', datetime.date(1988, 11, 5), 'dian@mail.com')
(5, 'Eko Saputra', 'Jl. Anggrek No. 9', datetime.date(1987, 3, 25), 'eko@mail.com')

Status peminjaman yang sudah lewat tanggal kembali telah diperbarui.

10 eksemplar buku 'Pemrograman Python' telah ditambahkan.

Data anggota yang tidak aktif selama lebih dari satu tahun telah dihapus.

Data Petugas:
(1, 'Andi Saputra', 'Admin', datetime.date(2022, 1, 1), '081234567890')
(2, 'Rina Wahyuni', 'Librarian', datetime.date(2023, 3, 15), '089876543210')

Judul Buku dan Pengarang (Tahun Terbit > 2015):
('Pemrograman Python', 'John Doe')
('Data Science', 'Jane Smith')
('Algoritma dan Struktur Data', 'Ahmad Zaki')
('Kecerdasan Buatan', 'Andi Setiawan')

Nama 

In [3]:
import mysql.connector
from mysql.connector import Error

# Membuat koneksi ke database MySQL
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='PerpustakaanDigital',  # Nama database
            user='root',                    # Username MySQL
            password=''             # Password MySQL
        )
        if connection.is_connected():
            print("Koneksi ke MySQL berhasil!")
        return connection

    except Error as e:
        print(f"Error saat terhubung ke MySQL: {e}")
        return None

# Fungsi untuk menutup koneksi
def close_connection(connection):
    if connection.is_connected():
        connection.close()
        print("Koneksi ke MySQL ditutup.")

# Fungsi untuk membuat tabel
def create_table(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Anggota (
                ID_Anggota INT PRIMARY KEY AUTO_INCREMENT,
                Nama VARCHAR(255),
                Alamat VARCHAR(255),
                Tanggal_Lahir DATE,
                Email VARCHAR(255)
            )
        """)
        connection.commit()
        print("Tabel Anggota berhasil dibuat.")
    except Error as e:
        print(f"Error saat membuat tabel: {e}")

# Fungsi untuk memasukkan data
def insert_data(connection):
    try:
        cursor = connection.cursor()
        sql_insert = """
            INSERT INTO Anggota (Nama, Alamat, Tanggal_Lahir, Email)
            VALUES (%s, %s, %s, %s)
        """
        data = [
            ('John Doe', 'Jl. Mawar No.1', '1990-01-01', 'john@example.com'),
            ('Jane Doe', 'Jl. Melati No.2', '1995-05-15', 'jane@example.com'),
            ('Ahmad', 'Jl. Kenanga No.3', '1992-03-25', 'ahmad@example.com')
        ]
        cursor.executemany(sql_insert, data)
        connection.commit()
        print(f"{cursor.rowcount} data berhasil dimasukkan.")
    except Error as e:
        print(f"Error saat memasukkan data: {e}")

# Fungsi untuk membaca data dari tabel
def read_data(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM Anggota")
        rows = cursor.fetchall()
        print("Data dalam tabel Anggota:")
        for row in rows:
            print(row)
    except Error as e:
        print(f"Error saat membaca data: {e}")

# Fungsi untuk mengupdate data
def update_data(connection, id_anggota, alamat_baru):
    try:
        cursor = connection.cursor()
        sql_update = "UPDATE Anggota SET Alamat = %s WHERE ID_Anggota = %s"
        data = (alamat_baru, id_anggota)
        cursor.execute(sql_update, data)
        connection.commit()
        print(f"Data ID_Anggota {id_anggota} berhasil diperbarui.")
    except Error as e:
        print(f"Error saat mengupdate data: {e}")

# Fungsi untuk menghapus data
def delete_data(connection, id_anggota):
    try:
        cursor = connection.cursor()
        # Perbarui foreign key di tabel Peminjaman
        cursor.execute("UPDATE Peminjaman SET ID_Anggota = NULL WHERE ID_Anggota = %s", (id_anggota,))
        # Hapus data dari tabel Anggota
        cursor.execute("DELETE FROM Anggota WHERE ID_Anggota = %s", (id_anggota,))
        connection.commit()
        print(f"Data ID_Anggota {id_anggota} berhasil dihapus.")
    except Error as e:
        print(f"Error saat menghapus data: {e}")

# Main program
if __name__ == "__main__":
    connection = create_connection()

    if connection is not None:
        # Membuat tabel jika belum ada
        create_table(connection)

        # Memasukkan data ke tabel
        insert_data(connection)

        # Membaca data dari tabel
        read_data(connection)

        # Mengupdate data anggota
        update_data(connection, 1, "Jl. Anggrek No.5")

        # Membaca data setelah update
        read_data(connection)

        # Menghapus data anggota
        delete_data(connection, 3)

        # Membaca data setelah penghapusan
        read_data(connection)

        # Menutup koneksi
        close_connection(connection)

Koneksi ke MySQL berhasil!
Tabel Anggota berhasil dibuat.
3 data berhasil dimasukkan.
Data dalam tabel Anggota:
(1, 'Ahmad Fauzi', 'Jl. Anggrek No.5', datetime.date(1985, 6, 10), 'ahmad@mail.com')
(2, 'Budi Putra', 'Jl. Melati No. 3', datetime.date(1990, 2, 18), 'budi@mail.com')
(4, 'Dian Sari', 'Jl. Kamboja No. 11', datetime.date(1988, 11, 5), 'dian@mail.com')
(5, 'Eko Saputra', 'Jl. Anggrek No. 9', datetime.date(1987, 3, 25), 'eko@mail.com')
(6, 'John Doe', 'Jl. Mawar No.1', datetime.date(1990, 1, 1), 'john@example.com')
(7, 'Jane Doe', 'Jl. Melati No.2', datetime.date(1995, 5, 15), 'jane@example.com')
(8, 'Ahmad', 'Jl. Kenanga No.3', datetime.date(1992, 3, 25), 'ahmad@example.com')
(9, 'John Doe', 'Jl. Mawar No.1', datetime.date(1990, 1, 1), 'john@example.com')
(10, 'Jane Doe', 'Jl. Melati No.2', datetime.date(1995, 5, 15), 'jane@example.com')
(11, 'Ahmad', 'Jl. Kenanga No.3', datetime.date(1992, 3, 25), 'ahmad@example.com')
(12, 'John Doe', 'Jl. Mawar No.1', datetime.date(1990, 1, 

In [9]:
import mysql.connector
from mysql.connector import Error

# Membuat koneksi ke database MySQL
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='PerpustakaanDigital',  # Nama database
            user='root',                    # Username MySQL
            password=''             # Password MySQL
        )
        if connection.is_connected():
            print("Koneksi ke MySQL berhasil!")
        return connection

    except Error as e:
        print(f"Error saat terhubung ke MySQL: {e}")
        return None

# Fungsi untuk menutup koneksi
def close_connection(connection):
    if connection.is_connected():
        connection.close()
        print("Koneksi ke MySQL ditutup.")

# Fungsi untuk membuat tabel Buku
def create_table_buku(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Buku (
                ID_Buku INT PRIMARY KEY AUTO_INCREMENT,
                Judul_Buku VARCHAR(255),
                Pengarang VARCHAR(100),
                Tahun_Terbit YEAR,
                Kategori VARCHAR(100),
                Jumlah INT
            )
        """)
        connection.commit()
        print("Tabel Buku berhasil dibuat.")
    except Error as e:
        print(f"Error saat membuat tabel Buku: {e}")

# Fungsi untuk memasukkan data ke tabel Buku
def insert_data_buku(connection):
    try:
        cursor = connection.cursor()
        sql_insert = """
            INSERT INTO Buku ( ID_BukJudul_Buku, Pengarang, Tahun_Terbit, Kategori, Jumlah)
            VALUES (%s, %s, %s, %s, %s)
        """
        data = [
            ('Pemrograman Python', 'Ali', 2023, 'Teknologi', 10),
            ('Dasar-Dasar SQL', 'Budi', 2022, 'Database', 15),
            ('Analisis Data', 'Citra', 2021, 'Data Science', 8)
        ]
        cursor.executemany(sql_insert, data)
        connection.commit()
        print(f"{cursor.rowcount} data Buku berhasil dimasukkan.")
    except Error as e:
        print(f"Error saat memasukkan data Buku: {e}")

# Fungsi untuk membaca data dari tabel Buku
def read_data_buku(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM Buku")
        rows = cursor.fetchall()
        print("Data dalam tabel Buku:")
        for row in rows:
            print(row)
    except Error as e:
        print(f"Error saat membaca data dari tabel Buku: {e}")

# Main program
if __name__ == "__main__":
    connection = create_connection()

    if connection is not None:
        # Membuat tabel Buku jika belum ada
        create_table_buku(connection)

        # Memasukkan data ke tabel Buku
        insert_data_buku(connection)

        # Membaca data dari tabel Buku
        read_data_buku(connection)

        # Menutup koneksi
        close_connection(connection)


Koneksi ke MySQL berhasil!
Tabel Buku berhasil dibuat.
Error saat memasukkan data Buku: 1062 (23000): Duplicate entry '0' for key 'PRIMARY'
Data dalam tabel Buku:
(1, 'Pemrograman Python', 'John Doe', 2018, 'Pemrograman', 15)
(2, 'Data Science', 'Jane Smith', 2020, 'Teknologi', 10)
(3, 'Algoritma dan Struktur Data', 'Ahmad Zaki', 2016, 'Pemrograman', 8)
(4, 'Kecerdasan Buatan', 'Andi Setiawan', 2019, 'Teknologi', 7)
(5, 'Jaringan Komputer', 'Budi Santoso', 2015, 'Teknologi', 6)
Koneksi ke MySQL ditutup.
